In [1]:
%cd ..

/home/antonbabenko/Projects/newsclass01


In [2]:
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%config InlineBackend.figure_format = "retina"

In [3]:
random_state = 42

In [4]:
data_dir = Path("data")
train_path = data_dir / "train.csv"
test_path = data_dir / "test_without_target.csv"

submission_dir = Path("submissions")
submission_path = submission_dir / "nlp" / "base_submission.csv"

In [5]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
train_df.head()

,Id,title,text,images,source
0,1,Міністри Зеленського лишають мільйони пацієнті...,Понад півтора мільйони пацієнтів із тяжкими ді...,NaN,5
1,2,В АМКУ звертаються щодо незаконної державної д...,За 15 років державна підтримка виробництва ене...,fd92a3dd-1109-49d8-8f5f-eeed72da22ef.png,6
2,5,У Польщі виявили майже 16 тисяч нових хворих н...,У Польщі виявили 15 698 нових хворих на корона...,f4a284d0-9bb7-4910-8a62-a7bfd0ec29b6.png,2
3,9,Закриття міжбанку: гривня трохи ослабла,2 жовтня котирування гривні до долара на закри...,NaN,3
4,10,"У Москві помер український шахіст із бази ""Мир...",У п'ятницю низка російських ЗМІ повідомили про...,"2cbb2339-6dd2-4000-a53c-225ec7aad892.png,d5233...",0


In [7]:
def preprocess(sentence):
    # Remove tags
    sentence = re.sub('<[^>]*>', '', sentence)

    # Substituting multiple spaces with single space
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)

    # Removing prefixed 'b'
    sentence = re.sub(r'^b\s+', '', sentence)

    # Converting to Lowercase
    sentence = sentence.lower()
    return sentence

In [8]:
labels = train_df.source.tolist()
sentences = [preprocess(sentence) for sentence in train_df.text.tolist()]

vectorizer = TfidfVectorizer(max_features=2500, min_df=7, max_df=0.8)
sentences = vectorizer.fit_transform(sentences).toarray()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=random_state)

In [10]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

/home/antonbabenko/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.82      0.71      0.76       353
           1       0.70      0.74      0.72       678
           2       0.95      0.74      0.83       213
           3       0.78      0.77      0.78       422
           4       0.84      0.90      0.87       548
           5       0.79      0.78      0.78       308
           6       0.77      0.80      0.78       714

    accuracy                           0.78      3236
   macro avg       0.81      0.78      0.79      3236
weighted avg       0.79      0.78      0.78      3236



In [12]:
test_sentences = [preprocess(sentence) for sentence in test_df.text.tolist()]
test_sentences = vectorizer.transform(test_sentences).toarray()

test_indeces = test_df.index.tolist()
y_test_preds = model.predict(test_sentences)

In [13]:
pd.DataFrame({"Id": test_indeces, "Predicted": y_test_preds}).to_csv(submission_path, index=False)